In [1]:
%%capture
# !pip install dkpro-cassis pdfplumber PyMuPDF stanza

In [2]:
# !pip install torch tensorflow sentencepiece transformers


In [24]:
import pandas as pd
from collections import defaultdict

# Step 1: Load data from Excel
file_path_excel = "2024-09-27_Food_Waren-und_Dienstleistungsgruppe_V_4.0.xlsx"
data_ids = pd.read_excel(file_path_excel, usecols=["Kriterium-ID"])["Kriterium-ID"].tolist()
data = pd.read_excel(file_path_excel, usecols=["Kategorie Kriterium", "Ausschreibungskriterium"])

# Step 2: Load and combine CSVs
file_paths = [
    "DATA/named_entity_annotations_1_Harmonisiert_ausschreibung.csv",
    "DATA/named_entity_annotations_2_Harmonisiert_ausschreibung.csv",
    "DATA/named_entity_annotations_4_Harmonisiert_ausschreibung.csv"
]
dataframes = [pd.read_csv(file_path) for file_path in file_paths]
combined_df = pd.concat(dataframes, ignore_index=True)

# Step 3: Process IDs and extract sentences
matched_ids = [
    id.replace("food-K-0", "K-").replace("food-HF-0", "K-").replace("K-0", "K-")
    for id in combined_df["criterion_id"].tolist()
]
sentences = combined_df["text"].tolist()

# Step 4: Initialize the result structure
result = {
    "sentencesss": [],
    "matched_guidlines": {
        "Ausschreibungskriterium": [],
        "Kategorie Kriterium": []
    }
}

# Match IDs and collect data
for i, matched_id in enumerate(matched_ids):
    for y, data_id in enumerate(data_ids):
        if matched_id == data_id:
            result["sentencesss"].append(sentences[i])
            result["matched_guidlines"]["Ausschreibungskriterium"].append(data.iloc[y]["Ausschreibungskriterium"])
            result["matched_guidlines"]["Kategorie Kriterium"].append(data.iloc[y]["Kategorie Kriterium"])

# Add false sentences to the result
false_sentences = {
    "sentences_False_1": [
        "Kitas und Tagesschulen nehmen Kinder ab Geburt bis 16 Jahren auf; aus allen sozialen Schichten, mit unterschiedlichster Herkunft, Kultur und Religion.",
        "Die Implementierung des Reglements «Gesunde Ernährung an den Bieler Betreuungsstrukturen» hat seinen Ursprung in einer städtischen Volksinitiative.",
        "Die Qualität soll über die Einführung der «Kalten Linie Biel» gewährleistet werden.",
        "Mittels einer zentralen Produktion werden reglementkonform die Mittagsmahlzeiten wie auch zum Teil die Zwischenmahlzeiten produziert.",
        "Die Standorte werden so eingerichtet, dass die Menüs mittels Steamer effizient und schonend regeneriert werden können.",
        "Je nach Entscheid kann es sein, dass die 5 Standorte, die aktuell über eine eigene Produktionsküche verfügen, als Übergangslösung weiterhin für sich selbst kochen.",
        "Eine Zusammenarbeit mit weiteren Partnern ist in der Offerte offen zu legen.",
        "Sie produzieren warm zu servierende Speisen zu 70% mittels «Cook and chill».",
        "Die anderen 30% verteilen sich auf Produktionsarten wie «Schockfrosten», «Sous-vide», «hot fill» oder Weiteres.",
        "Sie liefern die nötige Anzahl Menüs pro Tag.",
        "Die Kühlkette muss gewährleistet sein; die Qualität der Nahrung darf sich durch den Transport nicht vermindern.",
        "Sie achten auf sparsamen Einsatz von Zucker und Salz.",
        "Als Mittagessen bieten Sie ein Hauptgericht bestehend aus: Eiweisskomponente, Stärkebeilage, Gemüse und/oder Salat.",
        "Für Kinder mit Allergien kochen Sie spezielle Mahlzeiten.",
        "Die Produktion folgt einer Ernährungsphilosophie auf Basis einer aktuellen wissenschaftlicher Grundlage.",
        "Die Jury wird aufgrund eines Rankings die Punkte vergeben.",
        "Der durchschnittliche Preis soll sich auf ein Jahr beziehen und die genannten Qualitätsanforderungen berücksichtigen.",
        "Die Standorte verfügen über eine festgelegte Infrastruktur, die eine effiziente Lieferung ermöglicht.",
        "Die Ausschreibung erfolgt nach dem offenen Verfahren gemäss Verordnung über das öffentliche Beschaffungswesen (ÖBV).",
        "Eine jährliche Überprüfung der Qualität wird gemäss Reglement vorgeschrieben."
    ],
    "sentences_False_2": [
        "Die Ausschreibungsunterlagen bilden in ihrer Gesamtheit die Grundlage für die Evaluation des Vertragspartners.",
        "Das vorliegende Pflichtenheft stellt dabei das Grunddokument der Ausschreibungsunterlagen dar.",
        "Die Justizvollzugsanstalt Lenzburg betreibt eine eigene Küche, die die Mahlzeiten für die Insassen zubereitet.",
        "Die benötigten Lebensmittel müssen teilweise extern eingekauft werden.",
        "Der laufende Vertrag endet per 31. Dezember 2022, weshalb die Leistung neu ausgeschrieben wird.",
        "Die Anlieferung der bestellten Ware erfolgt innert 3 Tagen, spätestens aber innert Wochenfrist.",
        "Die Lieferzeiten sind Montag bis Freitag, jeweils von 07:30 bis 10:30 Uhr und von 13:30 bis 15:00 Uhr.",
        "Mangelhafte Waren hat der Anbieter umgehend zu ersetzen.",
        "Die Vergabestelle garantiert kein Auftragsvolumen für die Fleischwaren-Lieferungen.",
        "Die Evaluation erfolgt durch Mitarbeitende des Amts für Justizvollzug und weitere Mitarbeitende des Kantons Aargau.",
        "Fehlende Angaben führen dazu, dass die Offerte nicht berücksichtigt wird.",
        "Die Offerten werden nach festgelegten Zuschlagskriterien bewertet.",
        "Die Bewertung erfolgt anhand der Kriterien Preis, Qualität, Nachhaltigkeit und Referenzen.",
        "Der Anbieter verpflichtet sich, die geltenden Arbeitsschutzbestimmungen einzuhalten.",
        "Die gelieferten Lebensmittel müssen in guter, handelsüblicher Qualität sein.",
        "Die Offerte ist in deutscher Sprache einzureichen.",
        "Die Vergabestelle behält sich das Recht vor, die Submission abzubrechen oder nur teilweise zu realisieren.",
        "Der Anbieter haftet für Schäden, die durch seine Hilfspersonen oder Subunternehmer verursacht werden."
    ],

    "sentences_False_4": [
        "Die Ausschreibung erfolgt nach den Bestimmungen der Interkantonalen Vereinbarung über das öffentliche Beschaffungswesen.",
        "Die Mittagsverpflegung wird für drei schulergänzende Betreuungseinrichtungen und zwei Kindertagesstätten bereitgestellt.",
        "Der Auftrag umfasst die Kaltanlieferung der Mahlzeiten mit anschließender Regeneration vor Ort.",
        "Die gelieferten Speisen sollen altersgerecht, abwechslungsreich und ernährungswissenschaftlich fundiert sein.",
        "Die Lieferung erfolgt an bis zu fünf Standorte von Montag bis Freitag.",
        "Der Vertrag tritt ab dem 1. Juni 2023 für eine Dauer von drei Jahren in Kraft.",
        "Die Option zur Verlängerung des Vertrags um zweimal ein Jahr besteht.",
        "Die Verpflegung wird während 38 Schulwochen an Betreuungseinrichtungen geliefert.",
        "Für die Kindertagesstätten erfolgt die Lieferung während 48 Wochen im Jahr.",
        "Der Anbieter hat sicherzustellen, dass die Kühlkette während des Transports eingehalten wird.",
        "Umweltgerechte Fahrzeuge mit niedrigem Treibstoffverbrauch sind zu verwenden.",
        "Die Angebotsplanung muss saisonale und regionale Produkte berücksichtigen.",
        "Ein jährliches Controlling stellt die Einhaltung der Qualitätsvorgaben sicher.",
        "Bei mangelhafter Vertragserfüllung behält sich die Vergabestelle das Recht zur außerordentlichen Kündigung vor.",
        "Die Vergabe erfolgt an den Anbieter mit der höchsten Punktzahl gemäß Zuschlagskriterien.",
        "Die Offertenöffnung ist nicht öffentlich und wird protokolliert.",
        "Fragen zur Ausschreibung sind schriftlich über die Plattform bis zum Stichtag einzureichen."
    ]
    }

# Function to add false sentences
def add_false_sentences(sentences):
    for sentence in sentences:
        result["sentencesss"].append(sentence)
        result["matched_guidlines"]["Ausschreibungskriterium"].append(None)
        result["matched_guidlines"]["Kategorie Kriterium"].append(None)

# Add all false sentences
print(len(result["sentencesss"]))

add_false_sentences(false_sentences["sentences_False_1"])
add_false_sentences(false_sentences["sentences_False_2"])
add_false_sentences(false_sentences["sentences_False_4"])
print(len(result["sentencesss"]))
# Step 5: Group by sentences
grouped_result = {
    "unique_sentences": []
}

# Dictionary to group sentences
sentence_dict = defaultdict(lambda: {"Ausschreibungskriterium": [], "Kategorie Kriterium": []})

# Populate the dictionary
# Populate the dictionary
# Populate the dictionary
for sentence, guideline in zip(result["sentencesss"], zip(result["matched_guidlines"]["Ausschreibungskriterium"], result["matched_guidlines"]["Kategorie Kriterium"])):
    ausschreibung, kategorie = guideline
    sentence_dict[sentence]["Ausschreibungskriterium"].append(ausschreibung)
    sentence_dict[sentence]["Kategorie Kriterium"].append(kategorie)

# Convert the dictionary to the desired structure without filtering
grouped_result["unique_sentences"] = [
    {
        "sentence": sentence,
        "Ausschreibungskriterium": criteria["Ausschreibungskriterium"],
        "Kategorie Kriterium": criteria["Kategorie Kriterium"]
    }
    for sentence, criteria in sentence_dict.items()
]

# Print the structured result
print("Grouped Result:")
print(len(grouped_result["unique_sentences"]))

import pandas as pd

# Convert grouped_result to a DataFrame
df = pd.DataFrame(grouped_result["unique_sentences"])

# Save the DataFrame to a CSV file
output_file = "grouped_result.csv"
df.to_csv(output_file, index=False, encoding='utf-8')

print(f"Grouped result saved to {output_file}")






45
100
Grouped Result:
85
Grouped result saved to grouped_result.csv
